In [12]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\autogluon_type_class"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

In [6]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\dy\train_data\result.csv")

In [7]:
dataset.columns

Index(['OBJECTID', 'DLMC', 'new_TZ', 'XMin', 'YMin', 'XMax', 'YMax',
       'Centroid_X', 'Centroid_Y', 'Riverdis',
       ...
       'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE',
       'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90'],
      dtype='object', length=400)

In [5]:

print(dataset.size)
dataset.dropna(inplace=True)
print(dataset.size)
dataset.columns


13031600
3854800


Index(['OBJECTID', 'DLMC', 'new_TZ', 'XMin', 'YMin', 'XMax', 'YMax',
       'Centroid_X', 'Centroid_Y', 'Riverdis',
       ...
       'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE',
       'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90'],
      dtype='object', length=400)

In [8]:
dataset.columns

Index(['OBJECTID', 'DLMC', 'new_TZ', 'XMin', 'YMin', 'XMax', 'YMax',
       'Centroid_X', 'Centroid_Y', 'Riverdis',
       ...
       'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE',
       'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90'],
      dtype='object', length=400)

In [9]:
dataset["TZ_label"] = dataset.new_TZ.astype("category").cat.codes

In [10]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [13]:
result = dataset.groupby('TZ_label', observed=True)["new_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层灰泥质黄色石灰土', 1: '中层砂泥质黄壤', 2: '厚层砂泥质黄壤', 3: '厚层红泥质黄壤', 4: '厚层黏质黄色石灰土', 5: '浅石灰泥田', 6: '浅石红泥田', 7: '潮泥田', 8: '潮砂泥田', 9: '石灰泥田', 10: '石红泥田', 11: '腐中层壤质黄色石灰土', 12: '腐中层壤质黑色石灰土', 13: '腐中层灰泥质黄壤', 14: '腐中层砂泥质黄壤', 15: '腐中层砂泥质黄壤性土', 16: '腐中层砾壤质酸性紫色土', 17: '腐中层砾壤质钙质紫色土', 18: '腐中层硅质黄壤', 19: '腐中层红泥质黄壤', 20: '腐厚层壤质黄色石灰土', 21: '腐厚层壤质黑色石灰土', 22: '腐厚层灰泥质黄壤', 23: '腐厚层砂泥质黄壤', 24: '腐厚层硅质黄壤', 25: '腐厚层红泥质黄壤', 26: '腐薄层壤质黄色石灰土', 27: '腐薄层灰泥质黄壤', 28: '腐薄层砂泥质黄壤', 29: '腐薄层砾砂质黑色石灰土', 30: '腐薄层硅质山地灌丛草甸土', 31: '腐薄层硅质黄壤', 32: '腐薄层红泥质黄壤', 33: '腐薄层黏质黄色石灰土', 34: '薄层壤质中性紫色土', 35: '薄层灰泥质黄色石灰土', 36: '薄层砂泥质黄壤', 37: '薄层砾壤质中性紫色土', 38: '薄层砾壤质黄色石灰土', 39: '薄层砾壤质黑色石灰土', 40: '薄层砾硅质黄壤', 41: '薄层硅质酸性粗骨土', 42: '轻漂白粉泥田', 43: '重漂砂泥田', 44: '重漂红泥田', 45: '青石灰泥田', 46: '黄浅白粉泥田', 47: '黄浅砂泥田', 48: '黄浅红泥田', 49: '黄浅鳝泥田', 50: '黄白粉泥田', 51: '黄砂泥田', 52: '黄红泥田', 53: '黄青白粉泥田', 54: '黄鳝泥田', 55: '黄黄砂泥田'}


In [14]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict.json', 'w') as f:
    json.dump(result, f)

In [26]:
# 筛选水稻土数据
sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘'])]

In [30]:
# 筛选非自然土数据
fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘'])]

In [34]:
# 筛选自然土数据
zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])]

In [40]:
sdt_data.shape,fzrt_data.shape,zrt_data.shape,dataset.shape,sdt_data.shape[0]+fzrt_data.shape[0]+zrt_data.shape[0]==dataset.shape[0]

((4079, 401), (1488, 401), (27012, 401), (32579, 401), True)

In [17]:
# 定义要检查的关键字列表
keywords = ['DEM','ndvi', 'PCA_0', 'Slope','Aspect', 'MRVBF','TopographicWetnessIndex', 'Mean',  'PH','PRE','SRA','TMP','VAP','WIND','PlanCurvature']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


160
['DEM_COUNT', 'DEM_AREA', 'DEM_MIN', 'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN', 'DEM_PCT90', 'Aspect_COUNT', 'Aspect_AREA', 'Aspect_MIN', 'Aspect_MAX', 'Aspect_RANGE', 'Aspect_MEAN', 'Aspect_STD', 'Aspect_SUM', 'Aspect_MEDIAN', 'Aspect_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA', 'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE', 'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM', 'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'RelativeSlopePosition_COUNT', 'RelativeSlopePosition_AREA', 'RelativeSlopePosition_MIN', 'RelativeSlopePosition_MAX', 'RelativeSlopePosition_RANGE', 'RelativeSlopePosition_MEAN', 'RelativeSlopePosition_STD', 'RelativeSlopePosition_SUM', 'RelativeSlopePosition_MEDIAN', 'RelativeSlopePosition_PCT90', 'Slope_COUNT', 'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN', 'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'TopographicWetnessIndex_COUNT', 'TopographicWetnessIndex_AR

In [46]:
# features = ['DEM', 'Mean', 'ndvi', 'PCA_0', 'LON', 'LAT', 'PH','DL','DZ']
# features = list(dataset.columns[1:-1])
sdt_features= ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y','Riverdis',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
fzrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
zrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
target = "TZ_label"

In [43]:
# 获取sdt数据集
sdt_data = sdt_data[sdt_features+[target]]

In [44]:
# 获取非自然土数据集
fzrt_data = fzrt_data[fzrt_features+[target]]

In [47]:
# 获取自然土数据集
zrt_data = zrt_data[zrt_features+[target]]

In [48]:
# 分割训练集和测试集
sdt_data_train,sdt_data_test = train_test_split(sdt_data, test_size=0.2, random_state=42)
fzrt_data_train,fzrt_data_test = train_test_split(fzrt_data, test_size=0.2, random_state=42)
zrt_data_train,zrt_data_test = train_test_split(zrt_data, test_size=0.2, random_state=42)

# data = data
label = "TZ_label"

In [50]:

hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


In [62]:
# 训练水稻土模型
sdt_model_path = os.path.join(model_path,'sdt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(sdt_model_path):
    os.makedirs(sdt_model_path)
else:
    print("文件夹已存在")
sdt_predictor = TabularPredictor(label=label,path=sdt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(sdt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\sdt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       46.52 GB / 63.81 GB (72.9%)
Disk Space Avail:   475

文件夹已存在


	0.6187	 = Validation score   (f1_weighted)
	1.86s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7855	 = Validation score   (f1_weighted)
	0.66s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8025	 = Validation score   (f1_weighted)
	0.77s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.761	 = Validation score   (f1_weighted)
	0.6s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7849	 = Validation score   (f1_weighted)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6106	 = Validation score   (f1_weighted)
	21.88s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestEntr': 0.7, 'RandomForestGini': 0.233, 'NeuralNetFastAI': 0.033, 'ExtraTreesEntr': 0.033}
	0.8084	 = Validation score   (f1_weighted)
	0.51s	 = T

In [52]:
# 获取最佳模型
sdt_best_model = sdt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
sdt_model_summary = sdt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.808382  f1_weighted       0.100924   6.689525                0.000000           0.526224            2       True          7
1     RandomForestEntr   0.802510  f1_weighted       0.034358   0.797805                0.034358           0.797805            1       True          3
2     RandomForestGini   0.785454  f1_weighted       0.032990   0.666999                0.032990           0.666999            1       True          2
3       ExtraTreesEntr   0.784923  f1_weighted       0.033577   0.614102                0.033577           0.614102            1       True          5
4       ExtraTreesGini   0.761013  f1_weighted       0.031898   0.583248                0.031898           0.583248            1       True          4
5      NeuralNetFastAI   0.61869

In [54]:
sdt_predictor.leaderboard(sdt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.957934,0.808382,f1_weighted,0.304388,0.100924,6.689525,0.000000,0.000000,0.526224,2,True,7
1,RandomForestEntr,0.957022,0.802510,f1_weighted,0.082898,0.034358,0.797805,0.082898,0.034358,0.797805,1,True,3
2,RandomForestGini,0.954569,0.785454,f1_weighted,0.095032,0.032990,0.666999,0.095032,0.032990,0.666999,1,True,2
3,ExtraTreesEntr,0.954559,0.784923,f1_weighted,0.100225,0.033577,0.614102,0.100225,0.033577,0.614102,1,True,5
4,ExtraTreesGini,0.951473,0.761013,f1_weighted,0.100313,0.031898,0.583248,0.100313,0.031898,0.583248,1,True,4
5,NeuralNetTorch,0.832459,0.610590,f1_weighted,0.017971,0.016602,21.334347,0.017971,0.016602,21.334347,1,True,6
6,NeuralNetFastAI,0.805196,0.618697,f1_weighted,0.026233,0.000000,4.084396,0.026233,0.000000,4.084396,1,True,1


In [56]:
sdt_predictor.leaderboard(sdt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.830886,0.802510,f1_weighted,0.066651,0.034358,0.797805,0.066651,0.034358,0.797805,1,True,3
1,WeightedEnsemble_L2,0.822884,0.808382,f1_weighted,0.248184,0.100924,6.689525,0.001566,0.000000,0.526224,2,True,7
2,RandomForestGini,0.814616,0.785454,f1_weighted,0.082494,0.032990,0.666999,0.082494,0.032990,0.666999,1,True,2
3,ExtraTreesEntr,0.806044,0.784923,f1_weighted,0.083154,0.033577,0.614102,0.083154,0.033577,0.614102,1,True,5
4,ExtraTreesGini,0.800533,0.761013,f1_weighted,0.083344,0.031898,0.583248,0.083344,0.031898,0.583248,1,True,4
5,NeuralNetTorch,0.585360,0.610590,f1_weighted,0.021017,0.016602,21.334347,0.021017,0.016602,21.334347,1,True,6
6,NeuralNetFastAI,0.577410,0.618697,f1_weighted,0.014318,0.000000,4.084396,0.014318,0.000000,4.084396,1,True,1


In [63]:
# 训练非自然土模型
fzrt_model_path = os.path.join(model_path,'fzrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(fzrt_model_path):
    os.makedirs(fzrt_model_path)
else:
    print("文件夹已存在")
fzrt_predictor = TabularPredictor(label=label,path=fzrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(fzrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\fzrt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       46.56 GB / 63.81 GB (73.0%)
Disk Space Avail:   47

文件夹已存在


	0.69	 = Validation score   (f1_weighted)
	0.67s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8125	 = Validation score   (f1_weighted)
	0.65s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8083	 = Validation score   (f1_weighted)
	0.67s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7831	 = Validation score   (f1_weighted)
	0.64s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7779	 = Validation score   (f1_weighted)
	0.65s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.7294	 = Validation score   (f1_weighted)
	12.34s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestGini': 0.4, 'NeuralNetFastAI': 0.2, 'RandomForestEntr': 0.2, 'ExtraTreesGini': 0.2}
	0.8243	 = Validation score   (f1_weighted)
	0.47s	 = Traini

In [58]:
# 获取最佳模型
fzrt_best_model = fzrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
fzrt_model_summary = fzrt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.824335  f1_weighted       0.113532   3.123552                0.000000           0.470672            2       True          7
1     RandomForestGini   0.812453  f1_weighted       0.031313   0.646460                0.031313           0.646460            1       True          2
2     RandomForestEntr   0.808346  f1_weighted       0.032090   0.665161                0.032090           0.665161            1       True          3
3       ExtraTreesGini   0.783149  f1_weighted       0.034922   0.646546                0.034922           0.646546            1       True          4
4       ExtraTreesEntr   0.777906  f1_weighted       0.035474   0.631087                0.035474           0.631087            1       True          5
5       NeuralNetTorch   0.72943

In [59]:
fzrt_predictor.leaderboard(fzrt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.945579,0.824335,f1_weighted,0.231314,0.113532,3.123552,0.000000,0.000000,0.470672,2,True,7
1,RandomForestGini,0.943829,0.812453,f1_weighted,0.048476,0.031313,0.646460,0.048476,0.031313,0.646460,1,True,2
2,RandomForestEntr,0.942939,0.808346,f1_weighted,0.068953,0.032090,0.665161,0.068953,0.032090,0.665161,1,True,3
3,ExtraTreesGini,0.939454,0.783149,f1_weighted,0.100222,0.034922,0.646546,0.100222,0.034922,0.646546,1,True,4
4,ExtraTreesEntr,0.938505,0.777906,f1_weighted,0.065684,0.035474,0.631087,0.065684,0.035474,0.631087,1,True,5
5,NeuralNetTorch,0.895765,0.729436,f1_weighted,0.016969,0.000000,12.446265,0.016969,0.000000,12.446265,1,True,6
6,NeuralNetFastAI,0.882253,0.690042,f1_weighted,0.013664,0.015207,0.694713,0.013664,0.015207,0.694713,1,True,1


In [60]:
fzrt_predictor.leaderboard(fzrt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.786400,0.808346,f1_weighted,0.065157,0.032090,0.665161,0.065157,0.032090,0.665161,1,True,3
1,RandomForestGini,0.755087,0.812453,f1_weighted,0.067573,0.031313,0.646460,0.067573,0.031313,0.646460,1,True,2
2,WeightedEnsemble_L2,0.751126,0.824335,f1_weighted,0.198917,0.113532,3.123552,0.000998,0.000000,0.470672,2,True,7
3,ExtraTreesEntr,0.742041,0.777906,f1_weighted,0.049962,0.035474,0.631087,0.049962,0.035474,0.631087,1,True,5
4,ExtraTreesGini,0.726666,0.783149,f1_weighted,0.065190,0.034922,0.646546,0.065190,0.034922,0.646546,1,True,4
5,NeuralNetTorch,0.587593,0.729436,f1_weighted,0.013143,0.000000,12.446265,0.013143,0.000000,12.446265,1,True,6
6,NeuralNetFastAI,0.573484,0.690042,f1_weighted,0.000000,0.015207,0.694713,0.000000,0.015207,0.694713,1,True,1


In [64]:
# 训练自然土模型
zrt_model_path = os.path.join(model_path,'zrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(zrt_model_path):
    os.makedirs(zrt_model_path)
else:
    print("文件夹已存在")
zrt_predictor = TabularPredictor(label=label,path=zrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(zrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\zrt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       46.57 GB / 63.81 GB (73.0%)
Disk Space Avail:   475

文件夹已存在


	0.8573	 = Validation score   (f1_weighted)
	12.91s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9591	 = Validation score   (f1_weighted)
	2.23s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9604	 = Validation score   (f1_weighted)
	4.45s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9478	 = Validation score   (f1_weighted)
	0.83s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9524	 = Validation score   (f1_weighted)
	0.94s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8873	 = Validation score   (f1_weighted)
	117.81s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesEntr': 0.368, 'RandomForestEntr': 0.316, 'ExtraTreesGini': 0.263, 'RandomForestGini': 0.053}
	0.9628	 = Validation score   (f1_weighted)
	0.9

In [ ]:
# 获取最佳模型
zrt_best_model = zrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
zrt_model_summary = zrt_predictor.fit_summary()

In [ ]:
zrt_predictor.leaderboard(zrt_data_train, silent=True)

In [ ]:
zrt_predictor.leaderboard(zrt_data_test,silent=True)

In [73]:
# 获取sdt模型特征重要性
sdt_feature_importance = sdt_predictor.feature_importance(data=sdt_data_train)
pd.DataFrame(sdt_feature_importance)

,importance,stddev,p_value,n,p99_high,p99_low
Centroid_X,0.006974,6.895960e-04,1.132250e-05,5,0.008394,0.005554
YMax,0.006184,9.743489e-04,7.155497e-05,5,0.008190,0.004178
XMin,0.006070,3.598381e-04,1.474840e-06,5,0.006811,0.005329
Centroid_Y,0.005442,1.482716e-03,6.004229e-04,5,0.008495,0.002389
PH_MEAN,0.005234,6.765441e-04,3.276812e-05,5,0.006627,0.003841
YMin,0.004873,1.178716e-03,3.806324e-04,5,0.007300,0.002446
XMax,0.004826,7.482294e-04,6.719153e-05,5,0.006366,0.003285
PRE_MEAN,0.003940,7.038586e-04,1.171942e-04,5,0.005389,0.002491
Riverdis,0.003376,1.109171e-03,1.217075e-03,5,0.005660,0.001093
SRA_MEAN,0.002981,1.154576e-03,2.233814e-03,5,0.005359,0.000604


In [72]:
# 获取fzrt模型特征重要性
fzrt_feature_importance = fzrt_predictor.feature_importance(data=fzrt_data_train)
pd.DataFrame(fzrt_feature_importance)

,importance,stddev,p_value,n,p99_high,p99_low
DLMC,0.078939,0.006294,0.000005,5,0.091898,0.065979
Centroid_Y,0.008404,0.001391,0.000087,5,0.011267,0.005541
SRA_MEAN,0.008329,0.000785,0.000009,5,0.009945,0.006713
PH_MEAN,0.007367,0.002087,0.000696,5,0.011664,0.003071
YMin,0.007116,0.001616,0.000298,5,0.010443,0.003788
YMax,0.006553,0.001655,0.000449,5,0.009960,0.003145
XMax,0.006083,0.000593,0.000011,5,0.007303,0.004863
Centroid_X,0.006054,0.001497,0.000414,5,0.009136,0.002972
WIND_MEAN,0.005427,0.000790,0.000052,5,0.007053,0.003801
PRE_MEAN,0.005405,0.001034,0.000153,5,0.007533,0.003277


In [71]:
# 获取zrt模型特征重要性
zrt_feature_importance = zrt_predictor.feature_importance(data=zrt_data_train)
pd.DataFrame(zrt_feature_importance)

,importance,stddev,p_value,n,p99_high,p99_low
XMax,0.010869,0.001650,0.000062,5,0.014266,0.007473
XMin,0.009612,0.002139,0.000276,5,0.014016,0.005207
Centroid_X,0.008769,0.001046,0.000024,5,0.010923,0.006615
YMin,0.007392,0.000690,0.000009,5,0.008812,0.005971
YMax,0.006796,0.000850,0.000029,5,0.008545,0.005046
Centroid_Y,0.006744,0.000863,0.000031,5,0.008522,0.004967
SRA_MEAN,0.004254,0.000891,0.000218,5,0.006089,0.002419
PRE_MEAN,0.002494,0.000606,0.000386,5,0.003741,0.001247
WIND_MEAN,0.002288,0.000719,0.001031,5,0.003769,0.000807
TMP_MEAN,0.002170,0.000473,0.000255,5,0.003145,0.001196
